<a href="https://colab.research.google.com/github/Ijezue/Console-based-Retrieval-Augmented-Generation-RAG-application/blob/main/Console_based_RAG_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**
In this notebook, we will:  
1. Train an AI model to understand your PDF files and store them in a retrievable knowledge base. 🧠  
2. Build an interactive chatbot that can answer your questions using the knowledge we just trained. 🤖  

Before we dive in, let’s make sure all the required libraries are installed! 🚀  

---


  



# **Install Required Libraries**
To get started, we need to install some libraries. Run the cell below to install all of them:

# **What Do These Libraries Do?**
Here’s a quick breakdown of the libraries we’re using and why they’re important:  

1. **langchain**  
    - A super cool library for building AI applications by chaining multiple components like embeddings, retrievers, and LLMs together.  

2. **chromadb**  
    - A library for managing and querying vector databases. It allows us to save and search through the knowledge base.  

3. **pypdf**  
    - Used for reading and extracting text from PDF files. It’s the bridge between your documents and the AI!  

4. **langchain_community**  
    - Provides community-built tools like document loaders, retrievers, and more, tailored for specific tasks.  

5. **sentence_transformers**  
    - Enables us to create embeddings—special AI-friendly numerical representations of text that the model can understand.  

6. **langchain_huggingface**  
    - Provides integration with Hugging Face models, letting us use pre-trained models for embeddings and more.  

7. **pyngrok**  
    - A simple way to expose your Colab notebook to the internet, which can be useful for hosting or interacting with APIs.  

8. **langchain-groq**  
    - Enables seamless integration with Groq’s LLaMA-based models, which are great for chatbot functionality.  

In [ ]:
!pip install langchain chromadb pypdf langchain_community sentence_transformers langchain_huggingface pyngrok streamlit langchain-groq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 7.5 MB/s eta 0:00

# **Training the AI Knowledge Base**
**Training the Knowledge Base**  

In this cell, we’re doing the heavy lifting to set up the AI knowledge base:  

1. **Load your documents**:  
   We’ll scan through your folder and grab all the PDF files. If we don’t find any, don’t worry—we’ll gently nudge you to add some PDFs. 📂  

2. **Split the documents**:  
   PDFs can be long and hard to digest, so we’ll break them into bite-sized chunks for easier processing. Think of it like chopping a giant sandwich into snackable pieces! 🥪  

3. **Create embeddings**:  
   Using a cool pre-trained model, we’ll turn the text chunks into mathematical vectors (fancy word for AI-friendly numbers). This helps the AI understand the meaning of the text. 📈  

4. **Save the knowledge base**:  
   All the processed data gets saved in a retrievable format, so we can load it again later without repeating the work. 💾  

Once this cell runs successfully, your documents will be AI-ready! 🎊  

---



In [ ]:
# Import necessary libraries
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


# Define constants for the directories and API keys
GROQ_API_KEY = ""  # Set your Groq API key here
PERSIST_DIRECTORY = "/content/chroma_db/"  # Directory to save the Chroma vector store
FOLDER_PATH = "/content/myfiles"  # Folder containing the PDF files to process

# Create a vector store or load if present
def load_or_create_vector_store(texts, embeddings):

    if os.path.exists(PERSIST_DIRECTORY):
        print("Loading existing vector store...")
        return Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings)

    print("Creating a new vector store...")
    return Chroma.from_documents(texts, embeddings, persist_directory=PERSIST_DIRECTORY)

# Function to load and split documents and returns list of split document texts
def load_documents(folder_path):

    all_texts = []
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Folder not found: {folder_path}")

    pdf_found = False  # Track if there are any PDF files
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_found = True  # Found at least one PDF file
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {filename}")

            # Load the PDF file
            loader = PyPDFLoader(file_path)
            try:
                documents = loader.load()
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                continue  # Skip this file and continue with others

            # Split the documents into smaller chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,  # Maximum size of each chunk
                chunk_overlap=200,  # Overlap between chunks
                length_function=len,  # Function to calculate chunk length
                separators=["\n\n", "\n", " ", ""]  # Separators for splitting
            )
            texts = text_splitter.split_documents(documents)
            all_texts.extend(texts)

    if not pdf_found:
        raise ValueError("No PDF files found in the specified folder. Knowledge base must contain PDF files.")

    return all_texts

# Main execution for loading and training
print("Loading documents and setting up embeddings...")

try:
    # Load documents from the folder
    texts = load_documents(FOLDER_PATH)

    # Create embeddings using a pre-trained Hugging Face model
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-distilbert-base-v4")

    # Load or create the vector store
    db = load_or_create_vector_store(texts, embeddings)
    db.persist()

    print("Documents loaded and embeddings set up successfully.")
except Exception as e:
    print(f"Error during setup: {e}")


Loading documents and setting up embeddings...
Processing file: CS 5368 Reading Assignment 3.pdf
Processing file: 5368 Machine Learning .pdf


Processing file: L1-Introduction.pdf
Processing file: L7-ASPsolver.pdf
Processing file: L9-dynamicDomains.pdf


Processing file: L2-knowledgebase.pdf


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating a new vector store...
Documents loaded and embeddings set up successfully.


<ipython-input-2-7ade23d47f85>:76: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


# **Chat with Your AI Assistant 🤖**
Now that your documents have been processed, it’s time to unleash the power of conversation!  

In the next cell:  
- We set up an AI chatbot powered by Groq’s **LLaMA model** 🦙, which is a brilliant conversational wizard.  
- The chatbot uses the knowledge base created earlier to retrieve relevant pieces of information and answer your questions. 💬  

💡 **How it works**:  
- You ask a question.  
- The chatbot searches its brain (aka the knowledge base) to find relevant context.  
- It gives you a clear and confident answer—or politely says, "I don't know" if it's unsure.  

🛑 **Pro Tips**:  
- Type **exit** when you’re done chatting to say goodbye to the AI.  
- If your input is empty or unclear, the bot will kindly ask you to try again.  

Go ahead, test it out! Type your questions like you’re talking to a really smart friend. 😊  

  


In [ ]:
# Initialize the chatbot model
llm = ChatGroq(model="llama-3.1-70b-versatile", api_key=GROQ_API_KEY)

# Define the prompt template for the chatbot
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer: """
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

# Setting up the Retrieval-Augmented Generation (RAG) pipeline
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(),  # Use the vector store for context retrieval
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

print("Type your questions below (type 'exit' to quit).")

# Start an interactive chatbot session
while True:
    try:
        prompt = input("User Input: ")  # Get user input
        if prompt.strip().lower() == "exit":  # Check if the user wants to quit
            print("Goodbye!")
            break

        # Handle empty input
        if not prompt.strip():
            print("Please enter a question.")
            continue

        # Query the chatbot and retrieve the response
        response = qa_chain({"query": prompt})
        print(f"Model Response: {response['result']}")
    except KeyError:
        print("Error: Unable to process your question. Please try again.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


Type your questions below (type 'exit' to quit).
User Input: explain what an ann is to a toddler


<ipython-input-3-aa0904d75f6b>:37: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": prompt})


Model Response: Imagine you have lots of toy blocks, and each block is like a little computer that can talk to the other blocks. An Artificial Neural Network (ANN) is like a special way of connecting all those blocks together.

Imagine you want to teach a block to recognize pictures of cats and dogs. You would show it lots of pictures of cats and dogs, and it would learn to say "oh, that's a cat!" or "that's a dog!" by talking to all the other blocks.

The blocks (or "neurons") in the ANN are connected by special lines (like the "arcs" or arrows in the picture). When one block gets a message, it talks to all its friends (the other blocks) and they all work together to figure out what the message means.

So, an ANN is like a team of blocks working together to solve problems and learn new things.
User Input: what is cnn and what is the formula of ann
Model Response: Based on the given context, I don't have enough information to provide a formula for ANN (Artificial Neural Network). Howev